TODO TODO
\newpage

In [14]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

# FACT언어의 인터프리터
환경에서 자신을 참조하는 재귀적 클로저를 활용하여 간소화된 의미규칙에 따라 FACT언어의 인터프리터를 하스켈로 작성해 보자.
우선, 문법구조를 나타내는 타입(`Ty`)과 식(`Expr`) 그리고 의미구조를 표현하기 위한 값(`Val`)과 환경(`Env`)을 아래와 같이 선언한다.

In [15]:
type Nm = String  -- 변수 이름은 문자열로
data Ty = Z | Arr Ty Ty  deriving (Eq,Ord)
data Expr = Var Nm                 -- x
          | Lam Nm (Maybe Ty) Expr -- (\x:t.e)
          | Rec Nm (Maybe Ty) Expr -- rec f:t (\x:t2.e)
          | App Expr Expr          -- (e1 e2)
          | Lit Int                -- n
          | Add Expr Expr          -- e1 + e2
          | If Expr Expr Expr      -- if e then e1 else e0
          deriving (Eq, Ord)

data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]

In [16]:
instance Show Ty where
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x (Just t) e) = showParen (p > 1) $
      showString ("\\lambda "++x++"{:}"++show t++".") . showsPrec 1 e
  showsPrec p (Lam x Nothing e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (Rec f (Just t) e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f++"{:}") . showsPrec 0 t .
      showString "\\, " . showsPrec 2 e
  showsPrec p (Rec f Nothing e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f) .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where -- 노트북에 디스플레이하는 인스턴스 선언
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Ty where
    display t = display [latex $ "$"++show t++"$"]

In [17]:
idExpr = Lam "x" Nothing (Var "x")
ttExpr = Lam "x" (Just Z) (Lam "y" (Just Z) (Var "x")) 
ffExpr = Lam "x" (Just Z) (Lam "y" (Just Z) (Var "y")) 
-- 람다식, 산술식과, 조건식이 모두 활용된 예시 프로그램
If (Add (Lit 3) (Add (Var "x") (Var "y")))
   ffExpr
   (App idExpr (Rec "f" Nothing ffExpr))

In [18]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,cl) = x++"{\\mapsto}"++show cl
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Val where
    display v = display [latex $ "$"++show v++"$"]

FACT언어의 인터프리터 함수 `eval`을 아래와 같이 같이 작성해 볼 수 있다.

In [28]:
ev :: Env -> Expr -> Val  -- 타입이 잘 맞는 식만 실행한다고 가정
ev env (Var x) = v
    where Just v = lookup x env
ev env e@(Lam _ _ _) = Cl e env
ev env (Rec f t e@(Lam _ _ _)) = undefined -- eval의 경우와 같이
ev env (App e1 e2) = eval ((x,v2):env1) e
    where Cl (Lam x _ e) env1 = eval env e1 
          v2                  = eval env e2 
ev _   (Lit n)     = Vi n
ev env (Add e1 e2) = Vi (n1 + n2)
    where Vi n1 = ev env e1
          Vi n2 = ev env e2
ev env (If e e1 e0) = if n/=0 then ev env e1
                              else ev env e0
    where Vi n = eval env e

재귀함수를 제외하면, 매개변수에 타입을 표기할 뿐이지 FAC언어와 실행 내용은 같다.
아래는 타입 표기만 추가된 \ref{chap:FunArithEval}장과 마찬가지 테스트의 실행 사례다.

In [29]:
e1 = Lam "x" (Just Z) . Lam "y" (Just Z) $ If (Var "x") (Var "x" `Add` Var "y")
                                              (Var "y" `Add` Var "y")
e2 = Lit 2
e3 = Lit 3

ev [] e1
ev [] e2

In [30]:
App e1 e2
eval [] (App e1 e2)

In [31]:
App (App e1 e2) e3
ev [] (App (App e1 e2) e3)

In [32]:
App (App e1 (Lit 0)) e3
ev [] (App (App e1 (Lit 0)) e3)

$~$\vspace*{-1em}\newline\indent
FACT언어에서는 0부터 주어진 자연수 $i$까지의 총합을 구하는 재귀함수를 아래와 같이 간단히 작성할 수 있다.

In [33]:
e4 = Rec "f" (Just $ Arr Z Z) .
         Lam "i" (Just Z) $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- rec f (\i.if i then i + f(i + -1) else i)

그런데, 아직 `eval`함수에는 재귀함수요약식에 대한 내용이 작성되어 있지 않고 `undefined`로 남겨져 있다.
그래서 다음을 실행하면 에러가 난다.

In [34]:
-- rec f (\i.if i then i + f(i + -1) else i) 100
ev [] (e4 `App` Lit 100) -- 100부터 0까지의 총합

: 

\noindent
연습문제로 재귀적인 구조의 클로저를 활용하는 간소화된 의미구조를 하스켈로 작성하여 인터프리터 함수 `eval`를 완성한 다음
아래와 같이 100까지의 합을 구하도록 위의 테스트를 다시 실행하여 예상대로 0부터 100까지의 합이 5050으로 정상적으로 계산되는지 확인해 보라.

# 타입검사
타입환경($\Gamma$)에서 FACT언어의 식($e$)이 특정 타입($\tau$)인지, 즉 $\Gamma \vdash e : \tau$인지
검사하는 함수 `check`을 타입규칙(그림 \ref{fig:FACTtyrules})에 따라 하스켈로 작성하면 다음과 같다.

In [27]:
type TEnv = [(Nm,Ty)] -- 타입환경

check :: TEnv -> Expr -> Ty -> Bool
check tenv (Var x)      t  = lookup x tenv == Just t
check tenv (Lam x (Just t2) e) (Arr t2' t)
                           = t2==t2' && check ((x,t2):tenv) e t
check tenv (Rec f (Just t') e) t  = t == t' && check ((f,t'):tenv) e t
check tevv (Lit _)      Z  = True
check tenv (Add e1 e2)  Z  = check tenv e1 Z && check tenv e2 Z
check tenv (If e e1 e0) t  = check tenv e Z && check tenv e1 t
                                            && check tenv e2 t
check _   _             _  = False

# 타입유추
타입환경 $\Gamma$와 FACT언어의 식 $e$가 주어졌을 때
$\Gamma \vdash e : \tau$를 만족하는 타입($\tau$)이 존재한다면
구체적으로 $\tau$가 무엇인지 계산하여 이를 포함하는 리스트(`[`$\tau$`]`)를 계산하고,
존재하지 않는다면 빈 리스트(`[]`)를 계산하는 타입유추 함수 `infer`를 아래와 같이 작성해 보자.

In [18]:
infer :: TEnv -> Expr -> [Ty]  -- 적절한 타입이 존재하지 않으면 빈 리스트
infer tenv (Var x)      = case lookup x tenv of Just t  -> [t]
                                                Nothing -> []
infer tenv (Lam x t2 e) = undefined
infer tenv (Rec f t e)  = undefined
infer tevv (Lit _)      = [Z]
infer tenv (Add e1 e2)  = undefined
infer tenv (If e e1 e0) = undefined

\section*{요점정리}
* TODO
* TODO

\section*{연습문제}
1. TODO
1. TODO

\section*{탐구과제}
1. TODO aaa TODO
1. TODO aaa TODO